# Backtest strategy


In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Step 1: Data Collection
def fetch_historical_data(pool_address, start_date, end_date):
    # Fetch swap and liquidity data from Uniswap subgraphs or BigQuery
    # Placeholder function - replace with actual implementation
    return pd.DataFrame()  # Placeholder

# Step 2: Data Processing
def preprocess_data(data):
    # Clean and preprocess the data
    # Placeholder function - replace with actual implementation
    return data

# Step 3: Model Integration
def load_trained_model(model_path):
    # Load the trained RL model
    # Placeholder function - replace with actual implementation
    return None

def calculate_fees_accrued(historical_data, tick_lower, tick_upper, current_date, liquidity_position):
    # Filter trades within the tick range and current date

    trades_in_range = historical_data[(historical_data['tick'] >= tick_lower) &
                                      (historical_data['tick'] <= tick_upper) &
                                      (historical_data['date'] == current_date)]

    # Sum up the fees from these trades
    total_fees = 0
    for _, trade in trades_in_range.iterrows():
        fee_tier = trade['fee_tier']  # The fee tier of the pool (e.g., 0.3%)
        amount_in = trade['amount_in']  # The amount of asset traded
        fee = amount_in * fee_tier
        total_fees += fee * (liquidity_position / trade['active_liquidity'])
    
    return total_fees

def calculate_impermanent_loss(initial_prices, current_prices, initial_liquidity_value):
    # Calculate the value of holding initial liquidity without providing it
    initial_value_hold = initial_liquidity_value

    # Calculate the value of the current liquidity position
    current_value = (initial_prices[0] * (initial_liquidity_value / initial_prices[1])) + (initial_prices[1] * (initial_liquidity_value / initial_prices[0]))

    # Impermanent loss is the difference between holding and providing liquidity
    il = initial_value_hold - current_value
    return il


def calculate_gas_costs(gas_price, gas_used):
    # Example gas cost calculation
    gas_cost = gas_price * gas_used
    return gas_cost

def calculate_liquidity_position(initial_amount_A, initial_amount_B, current_price_A, current_price_B, tick_lower, tick_upper):
    # Simplified example, replace with actual liquidity calculation based on Uniswap V3 formulae
    if current_price_A >= tick_lower and current_price_A <= tick_upper:
        liquidity_A = initial_amount_A / current_price_A
    else:
        liquidity_A = 0

    if current_price_B >= tick_lower and current_price_B <= tick_upper:
        liquidity_B = initial_amount_B / current_price_B
    else:
        liquidity_B = 0

    liquidity_position = liquidity_A + liquidity_B
    return liquidity_position

# Step 4: Simulation Setup
def initialize_simulation(start_date, end_date, starting_capital):
    # Initialize simulation parameters
    capital = starting_capital
    current_date = start_date
    return capital, current_date

# Step 5: Rebalancing Logic
def get_inference(ddpg_agent_path, ppo_agent_path):
    #url = "http://49.213.164.57:40047/train_ddpg/"
    url = "http://127.0.0.1:8000/inference/"
    data = {
        "pool_id": "0x4e68ccd3e89f51c3074ca5072bbac773960dfa36",
        "ddpg_agent_path": ddpg_agent_path,
        "ppo_agent_path": ppo_agent_path
    }
    response = requests.post(url, json=data)
    inference_result = response.json()
    ddpg_action=inference_result['ddpg_action']
    ppo_action=inference_result['ppo_action']
    return ddpg_action,ppo_action

def simulate_rebalancing(capital, tick_lower, tick_upper, historical_data, current_date,liquidity_position,initial_liquidity_value,gas_used,gas_price,current_prices,initial_prices):
        fees_accrued = calculate_fees_accrued(historical_data, tick_lower, tick_upper, current_date, liquidity_position)
        il = calculate_impermanent_loss(initial_prices, current_prices, initial_liquidity_value)
        gas_costs = calculate_gas_costs(gas_price, gas_used)
        pnl = calculate_pnl(fees_accrued, il, gas_costs)
        trade_fee = 0.003 * capital  # Example: 0.3% trade fee
        slippage = 0.001 * capital  # Example: 0.1% slippage
        new_capital = capital + pnl - trade_fee - slippage
        return new_capital, trade_fee, slippage, fees_accrued, il, gas_costs, pnl
# Step 6: PNL Calculation
def calculate_pnl(fees_accrued, il, gas_costs):
    pnl = fees_accrued - il - gas_costs
    return pnl

# Step 7: Visualization and Reporting
def plot_results(positions, pnl_over_time):
    dates, tick_lowers, tick_uppers, capitals = zip(*positions)
    pnl_dates, pnls = zip(*pnl_over_time)
    
    plt.figure(figsize=(14, 7))
    plt.plot(dates, tick_lowers, label='Tick Lower')
    plt.plot(dates, tick_uppers, label='Tick Upper')
    plt.legend()
    plt.title('Positions Over Time')
    plt.xlabel('Date')
    plt.ylabel('Ticks')
    plt.show()
    
    plt.figure(figsize=(14, 7))
    plt.plot(pnl_dates, pnls, label='PNL')
    plt.legend()
    plt.title('PNL Over Time')
    plt.xlabel('Date')
    plt.ylabel('PNL')
    plt.show()

# Main Backtesting Loop
def backtest_ilp(start_date, end_date, pool_address, ddpg_agent_path, ppo_agent_path, starting_capital, rebalancing_frequency):
    # Step 1: Fetch historical data
    historical_data = fetch_historical_data(pool_address, start_date, end_date)
    historical_data = preprocess_data(historical_data)
    
    # Step 2: Initialize simulation
    capital, current_date = initialize_simulation(start_date, end_date, starting_capital)
    
    positions = []
    pnl_over_time = []
    initial_prices = [0, 0]  # Replace with actual initial prices
    current_prices = [0, 0]  # Replace with actual current prices
    initial_liquidity_value = starting_capital
    gas_price = 100  # Placeholder gas price
    gas_used = 500000  # Placeholder gas used
    
    while current_date <= end_date:
        
        # Step 3: Predict new positions
        ddpg_action, ppo_action = get_inference(ddpg_agent_path, ppo_agent_path)
        
        # Step 4: Rebalance portfolio
        new_capital, trade_fee, slippage, fees_accrued, il, gas_costs, pnl = simulate_rebalancing(capital, ddpg_action['price_lower'], ddpg_action['price_upper'], historical_data, current_date,liquidity_position, initial_liquidity_value,gas_price,gas_used,current_prices,initial_prices)
        
        # Step 5: Record position and PNL
        positions.append((current_date, ddpg_action['price_lower'], ddpg_action['price_upper'], new_capital))
        pnl = calculate_pnl(fees_accrued=0, il=0, gas_costs=trade_fee + slippage)  # Placeholder values
        pnl_over_time.append((current_date, pnl))
        
        # Move to the next rebalancing date
        capital = new_capital
        current_date += pd.Timedelta(days=rebalancing_frequency)
    
    # Step 6: Plot results
    plot_results(positions, pnl_over_time)
    
    return positions, pnl_over_time

# Example usage
positions, pnl_over_time = backtest_ilp(
    start_date=pd.Timestamp('2022-12-01'),
    end_date=pd.Timestamp('2022-12-31'),
    pool_address='0xcbcdf9626bc03e24f779434178a73a0b4bad62ed',
    ddpg_agent_path='model_storage/ddpg/ddpg_1',
    ppo_agent_path='model_storage/ppo/lstm_actor_critic_batch_norm',
    starting_capital=100000,
    rebalancing_frequency=7  # Rebalance every 7 days
)


NameError: name 'liquidity_position' is not defined

# Get historical data

In [2]:
import pandas as pd
import datetime as dt
import requests
import json

def fetch_swaps_data(pool_address, start_date, end_date):
    # Fetch swap events from TheGraph or another API
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
    start_date = dt.datetime.strptime(start_date, '%d-%m-%y')
    end_date = dt.datetime.strptime(end_date, '%d-%m-%y')
    
    # Convert datetime objects to timestamps
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())
  
    query = f"""
    {{
      swaps(
        where: {{
          pool: "{pool_address}",
          timestamp_gt: "{start_timestamp}",
          timestamp_lt: "{end_timestamp}"
        }}
      ) {{
        id
        timestamp
        tick
        amount0
        amount1
        amountUSD
        sqrtPriceX96
        tick
      }}
    }}
    """
    # Placeholder: Replace with actual API call
    response = requests.post(url, json={'query': query})

    # Get the JSON data from the response
    data = json.loads(response.text)

    swaps_data = data['data']['swaps']

    # Convert to DataFrame
    df = pd.json_normalize(swaps_data)
    # Extracting data from response
    swaps_data = pd.DataFrame(swaps_data)
    #print(swaps_data)
    return swaps_data

def fetch_liquidity_data(pool_address, start_date, end_date):
    # Fetch liquidity data from TheGraph or using a web3 client
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
    start_date = dt.datetime.strptime(start_date, '%d-%m-%y')
    end_date = dt.datetime.strptime(end_date, '%d-%m-%y')
    
    # Convert datetime objects to timestamps
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())
    query = """
    {{
      poolHourDatas(
        where: {{
          pool: "{pool_address}",
          periodStartUnix_gt: {start_timestamp},
          periodStartUnix_lt: {end_timestamp}
        }}
      ) {{
        periodStartUnix
        liquidity
        tick
        sqrtPrice
      }}
    }}
    """.format(pool_address=pool_address, start_timestamp=start_timestamp, end_timestamp=end_timestamp)
     
    response = requests.post(url, json={'query': query})

    # Get the JSON data from the response
    data = json.loads(response.text)
    #print(data)
    data = data['data']['poolHourDatas']

    # Convert to DataFrame
    df = pd.json_normalize(data)
    # Extracting data from response
    liquidity_data = pd.DataFrame(data)
    return liquidity_data

def preprocess_data(swaps_data, liquidity_data):
    # Convert timestamps to datetime
    swaps_data['timestamp'] = pd.to_datetime(swaps_data['timestamp'], unit='s')
    liquidity_data['periodStartUnix'] = pd.to_datetime(liquidity_data['periodStartUnix'], unit='s')
    swaps_data = swaps_data.sort_values('timestamp')
    liquidity_data = liquidity_data.sort_values('periodStartUnix')
    
    # Merge dataframes
    merged_data = pd.merge_asof(swaps_data.sort_values('timestamp'), liquidity_data.sort_values('periodStartUnix'),
                                left_on='timestamp', right_on='periodStartUnix', direction='backward')
    
    merged_data = merged_data.rename(columns={'liquidity': 'active_liquidity'})
    return merged_data

In [3]:
pool_id="0xcbcdf9626bc03e24f779434178a73a0b4bad62ed"
start_date="21-5-24"
end_date="21-6-24"
swaps_data=fetch_swaps_data(pool_id,start_date,end_date)
liquidity_data=fetch_liquidity_data(pool_id,start_date,end_date)
merged_data=preprocess_data(swaps_data,liquidity_data)

In [4]:
def calculate_fees_accrued(merged_data, tick_lower, tick_upper, liquidity_position, fee_tier):
    total_fees = 0
    for _, trade in merged_data.iterrows():
        if tick_lower <= trade['tick'] <= tick_upper:
            fee = trade['amountUSD'] * fee_tier
            total_fees += fee * (liquidity_position / trade['active_liquidity'])
    return total_fees

def calculate_impermanent_loss(initial_value, current_value):
    return initial_value - current_value

def calculate_gas_costs(gas_price, gas_used):
    return gas_price * gas_used

def calculate_pnl(fees_accrued, il, gas_costs):
    return fees_accrued - il - gas_costs

In [5]:
from UNI_v3_funcs import budget_to_liquidity, price_to_valid_tick
ddpg_agent_path='model_storage/ddpg/ddpg_1'
ppo_agent_path='model_storage/ppo/lstm_actor_critic_batch_norm'
fee_tier=0.03

ddpg_action,ppo_action=get_inference(ddpg_agent_path,ppo_agent_path)
liquidity_position=budget_to_liquidity(tick_lower=price_to_valid_tick(ddpg_action['price_lower']),tick_upper=price_to_valid_tick(ddpg_action['price_upper']),usd_budget=10000)
calculate_fees_accrued(merged_data,ddpg_action['price_lower'],ddpg_action['price_upper'],liquidity_position,fee_tier)

NameError: name 'self' is not defined

# Calculate IL

In [1]:

import UNI_v3_funcs as liq_amounts
import numpy as np
import pandas as pd

#Pool features
decimals0=6
decimals1=18
tick_space=60 #fee_tier
test=pd.DataFrame()

#Ticks of the range to simulate
tick_lower=196740
tick_upper=201120
#Tick of the pool at time0
tick_entry=198740
sqrt_entry=(1.0001**(tick_entry/2)*(2**96))

#Create vector with every tick_space in the range


test['sqrt']=(1.0001**(test['ticks']/2)*(2**96))
test['sqrtA']=(1.0001**(tick_lower/2)*(2**96))
test['sqrtB']=(1.0001**(tick_upper/2)*(2**96))

#Calls liquidity amount formula to calculate the relation of tokens required to pool at time0
relation_token= abs(liq_amounts.amounts_relation (tick_entry,tick_lower,tick_upper,decimals0,decimals1))

#Calculating amounts of token0 based on 1 unit of t1
amount1=1
amount0=1/relation_token

#Calls amountsforliquidity formula to get liquidity deployed 
initial_liquidity= liq_amounts.get_liquidity(tick_entry,tick_lower,tick_upper,amount0,amount1,decimals0,decimals1)   
test['liquidity']=initial_liquidity
#Calculate amounts at every tick space based on liquidity
test['amounts_0']=test.apply(lambda x: liq_amounts.get_amounts(x['ticks'],tick_lower,tick_upper,x['liquidity'],decimals0,decimals1)[0] ,axis=1)
test['amounts_1']=test.apply(lambda x: liq_amounts.get_amounts(x['ticks'],tick_lower,tick_upper,x['liquidity'],decimals0,decimals1)[1] ,axis=1)

#Calculating price at every tick_space
test['price_t1/t0']=1/(1.0001**test['ticks']/10**(decimals1-decimals0))

#Calculating the value of the position in every tick_space
test['value_LP']=test['amounts_1']*test['price_t1/t0']+test['amounts_0']

#Calculating the value of holding the initial amount of tokens
test['value_Hold']=amount1*test['price_t1/t0']+amount0

#Calculating IL
test['IL_USD']=test['value_LP']-test['value_Hold']
test['ILvsHold']=test['IL_USD']/test['value_Hold']

#Plotting price vs IL
import matplotlib.pyplot as plt
plt.plot(test['price_t1/t0'],test['IL_USD'])
# plt.plot(test['price_t1/t0'],test['ILvsHold'])
plt.show()



KeyError: 'ticks'

# Price Impact

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun May 23 10:11:41 2021

@author: Julian
"""


from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import datetime
import math


from scr_common import UNI_v3_funcs as UNI_funcs

url='https://api.thegraph.com/subgraphs/name/ianlapham/uniswap-v3-prod'

query='''query pl($pool_id:String) 
    {pools(where: { id: $pool_id } ) 
        {
        token0 {symbol decimals}
        token1 {symbol decimals}
        feeTier
        feesUSD
        volumeUSD
        token0Price
        token1Price
        volumeToken0
        volumeToken1
        tick
        liquidity
        sqrtPrice
        ticks(first:500,skip:6) {id,price0,price1,tickIdx,liquidityGross,liquidityNet,volumeUSD}
        }
        }'''



'''GraphQl'''
def query_univ3(url,query_a,params):

    sample_transport=RequestsHTTPTransport(
       url=url,
       verify=True,
       retries=5,)
    client = Client(transport=sample_transport)
    query = gql(query_a)
    response = client.execute(query,variable_values=params)
    
    
    return response


def get_liquidity(pool_id,price_range):

    # Falta usar los decimales que nos da subgraph para cada token

    '''Calculating liquidity and amounts of token by tick'''
    
    # Query subgraph
    params ={"pool_id":pool_id}   
    a=query_univ3(url,query,params)
    # create df with pool_data and tick_data
    pool_data=a['pools'][0]
    tick_data=pd.io.json.json_normalize(a['pools'][0]['ticks'])
    del pool_data['ticks']

    # Making up pool_data not related to ticks
    decimal0=int(pool_data['token0']['decimals'])
    decimal1=int(pool_data['token1']['decimals'])
    pool_data['liquidity']=int(pool_data['liquidity'])
    pool_data['sqrtPrice']=int(pool_data['sqrtPrice'])
    pool_data['tick']=int(pool_data['tick'])
    # Calculating actual price
    pool_data['Price']=1.0001**pool_data['tick']/10**(decimal1-decimal0)
    
    # Making up tick_data 
    tick_data['tickIdx']=tick_data['tickIdx'].astype(int)
    tick_data['price1']=1.0001**tick_data['tickIdx']/10**(decimal1-decimal0)
    tick_data['price0']=1/tick_data['price1']
    tick_data['liquidityNet']=pd.to_numeric(tick_data['liquidityNet'],errors='coerce',downcast='signed')
    tick_data['liquidityGross']=pd.to_numeric(tick_data['liquidityGross'],errors='coerce',downcast='signed')

    tick_data=tick_data.sort_values(by=['tickIdx'], ascending=True)

    # Calculating closing ticks to actual price and liquidity on active tick
    tick_space= int(pool_data['feeTier']) *2 /100 
    closest_ticks=(math.floor(pool_data['tick']/tick_space)*tick_space, math.ceil(pool_data['tick']/tick_space)*tick_space)
    pool_data['tickUpper']=closest_ticks[1]
    pool_data['tickLower']=closest_ticks[0]
    active_amounts=UNI_funcs.get_amounts(pool_data['tick'],closest_ticks[0],closest_ticks[1],pool_data['liquidity'],decimal0,decimal1)
    T1_onTick=active_amounts[1]+active_amounts[0]*float(pool_data['token1Price'])
    T0_onTick=active_amounts[1]*float(pool_data['token0Price'])+active_amounts[0]

    # Creating pa,pb range for a tick taking in consideration the direction of the pool
    # As uniswap UI paint as active zone till Upper tick of the active we split df like that
    tick_data['tickIdx_B']= np.where( tick_data['tickIdx']<=closest_ticks[1], tick_data['tickIdx'].shift(1), tick_data['tickIdx'].shift(-1)  )

    

    # Creating 2 tick_datas, one for token0 side, another for token1 side
    # As uniswap UI paint as active zone till Upper tick of the active we split df like that
    tick_data_token0=tick_data.loc[tick_data['tickIdx']>closest_ticks[1]]
    tick_data_token1=tick_data.loc[tick_data['tickIdx']<=closest_ticks[1]].sort_values(by=['tickIdx'], ascending=False)
    
    # Liquidity in each tick is equal to Activeliquidity +/- the rolling liquidityNet
    tick_data_token0['rolling_Net'] = tick_data_token0['liquidityNet'].rolling(100000, min_periods=1).sum()
    tick_data_token0=tick_data_token0.dropna()
    tick_data_token0['liquidity'] = pool_data['liquidity'] + tick_data_token0['rolling_Net']
    #Calculate for Liquitidy in each tick the amount of tokens
    tick_data_token0['amounts_Rolling'] = tick_data_token0.apply(lambda x: UNI_funcs.get_amounts(pool_data['tick'],int(x['tickIdx']),int(x['tickIdx_B']),int(x['liquidity']),decimal0,decimal1) ,axis=1)
    # Extract amount token 0 and amount token 1
    tick_data_token0['amount0']= [x[0] for x in tick_data_token0['amounts_Rolling']]
    tick_data_token0['amount1']= [x[1] for x in tick_data_token0['amounts_Rolling']]
    # Accumulated liquidity on amounts
    tick_data_token0['amount0_ac']=tick_data_token0['amount0'].cumsum()
    tick_data_token0['amount1_ac']=tick_data_token0['amount1'].cumsum()
    del tick_data_token0['amounts_Rolling']
    
    # Liquidity side 2 (As include the active zone some additional fix has to be made)
    tick_data_token1['rolling_Net'] = tick_data_token1['liquidityNet'].rolling(100000, min_periods=1).sum()
    tick_data_token1=tick_data_token1.dropna()
    tick_data_token1['liquidity'] = pool_data['liquidity']-tick_data_token1['rolling_Net']
    # Fix data on active zone
    tick_data_token1['liquidity'] = np.where(tick_data_token1['tickIdx']==closest_ticks[1] ,pool_data['liquidity'],tick_data_token1['liquidity'])
    tick_data_token1=tick_data_token1.sort_values(by=['tickIdx'])
    #Calculate for Liquitidy in each tick the amount of tokens
    tick_data_token1['amounts_Rolling'] = tick_data_token1.apply(lambda x: UNI_funcs.get_amounts(pool_data['tick'],int(x['tickIdx']),int(x['tickIdx_B']),int(x['liquidity']),decimal0,decimal1) ,axis=1)
    # Extract amount token 0 and amount token 1
    tick_data_token1['amount0']= [x[0] for x in tick_data_token1['amounts_Rolling']]
    tick_data_token1['amount1']= [x[1] for x in tick_data_token1['amounts_Rolling']]
    # Get amounts on the active zone and put it on the df
    tick_data_token1['amount0'] = np.where( tick_data_token1['tickIdx']==closest_ticks[1], active_amounts[0],tick_data_token1['amount0'])    
    tick_data_token1['amount1'] = np.where( tick_data_token1['tickIdx']==closest_ticks[1] ,active_amounts[1],tick_data_token1['amount1'])    
    # Accumulated liquidity on amounts (should be done on a inverse way)
    tick_data_token1['amount0_ac'] =tick_data_token1['amount0']
    tick_data_token1['amount1_ac'] = tick_data_token1.loc[::-1, 'amount1'].cumsum()[::-1]
    del tick_data_token1['amounts_Rolling']

    # Concatenate the two tick_datas and preparing data to be saved
    final_tick_data=pd.concat([tick_data_token1,tick_data_token0])
    final_tick_data['timestamp']=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')  

    # Filter to a reasonable distance from price 
    lower_filter = closest_ticks[0]-price_range*(1/0.0001)
    upper_filter = closest_ticks[1]+price_range*(1/0.0001)
    final_tick_data=final_tick_data.loc[(final_tick_data['tickIdx']<= upper_filter ) & (final_tick_data['tickIdx']>=lower_filter )]


    final_tick_data['geometric_mean'] = (final_tick_data['tickIdx']*final_tick_data['tickIdx_B'])**(1.0/2)
    final_tick_data['price1_mean']=1.0001**final_tick_data['geometric_mean']/10**(decimal1-decimal0)
    final_tick_data['price0_mean']=1/final_tick_data['price1_mean']    
    

    return final_tick_data,pool_data




def get_tradeImpact(pool_id,tokenIn,qtyIn):
    
    # Fetching actual state of the pool liquidity
    data = get_liquidity(pool_id,0.6) 
    tickData = data[0]  
    poolData = data[1] 
    
    feeTier=int(poolData['feeTier'])/1000000
    tick_space=int(poolData['feeTier'])*2/100
    decimal0=int(poolData['token0']['decimals'])
    decimal1=int(poolData['token1']['decimals'])

    qtyIn = [x*feeTier for x in qtyIn]

    final_prices={}
    # Looping through different swap size
    for qty in qtyIn:
        
        # In this case swap cross tick zones                
        if tokenIn == 1:
            # Filtering liquidity side to use
            # When token 1 is being swapped liquidity move from tick lower to tick upper
            tickData=tickData.loc[tickData['tickIdx'] >=poolData['tickUpper']]

            # Calculate qty of token 0 available in every zone
            # Fixing geometric mean in active zone (simplification of geometric mean as for a tick is more or less 1/2 of the difference)
            tickData['geometric_mean'] = np.where ( (tickData['amount0']>0) & (tickData['amount1']>0),
                                                    (tickData['tickIdx'].iloc[0]+poolData['tick'])/2,
                                                    (tickData['tickIdx']+tickData['tickIdx_B'])/2)    
            tickData['price1_mean']=1.0001**tickData['geometric_mean']/10**(decimal1-decimal0)
            tickData['price0_mean']=1/tickData['price1_mean']    
            
            # token1 will be swapped with price = geometric mean inside a tick zone
            tickData['amount1_available'] = tickData['amount0'] / tickData['price0_mean']
            tickData['amount1_available_ac'] = tickData['amount1_available'].cumsum()
            
            # Checking if swap cross active zone
            if qty > tickData['amount1_available_ac'].iloc[0]:
                # Getting active zone after swap
                tickData['final_zone'] =  qty < tickData['amount1_available_ac']
                tickData['final_zone'] = np.where((tickData['final_zone'] == True) & (tickData['final_zone'].shift(1) == False),1,0)
                zone_tickUpper = tickData.loc[tickData['final_zone']==1]['tickIdx_B']
                zone_tickLower = tickData.loc[tickData['final_zone']==1]['tickIdx']
                # qty that remains on the active zone
                already_swapped_idx = tickData[tickData['final_zone'] == 1].index.values.astype(int)[0]-1 
                qty_onActive =  qty - int(tickData.loc[tickData.index==already_swapped_idx]['amount1_available_ac'] ) # qty - already swapped qty 
                # Final tick after swapping
                final_tick = int(zone_tickLower + math.ceil( (qty_onActive/int(tickData.loc[(tickData['final_zone'] == 1)]['amount1_available']) ) *tick_space))
                initial_tick = poolData['tick']
                print(final_tick)
            
            else:
                # In this case swap doesnt cross tick zones
                initial_tick = poolData['tick']
                final_tick = int(initial_tick + math.ceil( (qty/int(tickData['amount1_available'].iloc[0] ) * tick_space )))
                print(final_tick)


        elif tokenIn == 0:
            
            # Filtering liquidity side to use
            # When token 0 is being swapped liquidity move from tick upper to tick lower
            tickData=tickData.loc[tickData['tickIdx'] <=poolData['tickUpper']].sort_values(by=['tickIdx'], ascending=False)
            tickData=tickData.reset_index(drop=True)
            # Calculate qty of token 1 available in every zone
            # Fixing geometric mean in active zone (simplification of geometric mean as for a tick is more or less 1/2 of the difference)
            tickData['geometric_mean'] = np.where ( (tickData['amount0']>0) & (tickData['amount1']>0),
                                                    (tickData['tickIdx'].iloc[0]+poolData['tick'])/2,
                                                    (tickData['tickIdx']+tickData['tickIdx_B'])/2)    

            tickData['price1_mean']=1.0001**tickData['geometric_mean']/10**(decimal1-decimal0)
            tickData['price0_mean']=1/tickData['price1_mean']   
            
            
            # token1 will be swapped with price = geometric mean inside a tick zone
            tickData['amount0_available'] = tickData['amount1'] / tickData['price1_mean']
            tickData['amount0_available_ac'] = tickData['amount0_available'].cumsum()

            # Checking if swap cross active zone
            if qty > tickData['amount0_available_ac'].iloc[0]:
                # Getting active zone after swap
                tickData['final_zone'] =  qty < tickData['amount0_available_ac']
                tickData['final_zone'] = np.where((tickData['final_zone'] == True) & (tickData['final_zone'].shift(1) == False),1,0)
                zone_tickUpper = tickData.loc[tickData['final_zone']==1]['tickIdx']
                zone_tickLower = tickData.loc[tickData['final_zone']==1]['tickIdx_B']
                # qty that remains on the active zone
                already_swapped_idx = tickData[tickData['final_zone'] == 1].index.values.astype(int)[0]-1 
                qty_onActive =  qty - int(tickData.loc[tickData.index==already_swapped_idx]['amount0_available_ac'] ) # qty - already swapped qty 
                # Final tick after swapping
                final_tick = int(zone_tickLower + math.ceil( (qty_onActive/int(tickData.loc[(tickData['final_zone'] == 1)]['amount0_available']) ) *tick_space))
                initial_tick = poolData['tick']
                print(final_tick)
            
            else:
                # In this case swap doesnt cross tick zones
                initial_tick = poolData['tick']
                final_tick = int(initial_tick - math.ceil( (qty/int(tickData['amount0_available'].iloc[0] ) * tick_space )))
                initial_tick = poolData['tick']
                print(final_tick)        

        # Dict with tick and both prices
        final_prices[qty] = {'ticks':(int(initial_tick),int(final_tick))}

        initial_price0= 1.0001**initial_tick/10**(int(poolData['token1']['decimals'])-int(poolData['token0']['decimals']))
        final_price0= 1.0001**final_tick/10**(int(poolData['token1']['decimals'])-int(poolData['token0']['decimals']))
        price0_swap= (initial_price0*final_price0)**(1/2)
        final_prices[qty].update({'price0':(initial_price0 ,final_price0)})
        final_prices[qty].update({'price0_swap': price0_swap }) # Geometric mean of range
        final_prices[qty].update({'price1':(1/initial_price0 ,1/final_price0)})
        final_prices[qty].update({'price1_swap':  1/price0_swap  })
        final_prices[qty].update({'price_impact(%)': round( (initial_price0 - final_price0) / initial_price0 * 100 ,2) })
        final_prices[qty].update({'price_impact_swap(%)': round( (initial_price0 - price0_swap) / initial_price0 * 100 ,2) })
        if tokenIn == 1:
            final_prices[qty].update({'qty_received': round( (qty/price0_swap) ,2) })
        elif tokenIn == 0:
            final_prices[qty].update({'qty_received': round( (qty/(1/price0_swap)) ,2) })

    return final_prices


# # Address to analyze
# pool_id = "0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801".lower()

# # Trades to analyze
# tokenIn = 0
# zeros_toTest = (1,5)
# qtyIn = [1*10**x for x in range(zeros_toTest[0],zeros_toTest[1]+1)]

# trade_impact = get_tradeImpact(pool_id,tokenIn,qtyIn)

# print(1)


ModuleNotFoundError: No module named 'gql'

In [3]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

def backtest_ilp(start_date, end_date, pool_address, ddpg_agent_path, ppo_agent_path, starting_capital, rebalancing_frequency):
    # Fetch historical data for the specified pool
    historical_data = fetch_historical_data(pool_address, start_date, end_date)
    
    # Initialize variables
    capital = starting_capital
    positions = []
    pnl_over_time = []
    current_date = start_date

    while current_date <= end_date:
        # Get the pool state for the current date
        pool_state = get_pool_state(historical_data, current_date)
        
        # Call the inference endpoint to get the new position
        tick_lower, tick_upper = get_inference(pool_state, ddpg_agent_path, ppo_agent_path,pool_address)
        
        # Simulate rebalancing and calculate trade fees and slippage
        capital, trade_fee, slippage = simulate_rebalancing(capital, tick_lower, tick_upper, historical_data, current_date)
        
        # Record position and PNL
        positions.append((current_date, tick_lower, tick_upper, capital))
        pnl_over_time.append((current_date, capital - starting_capital - trade_fee - slippage))
        
        # Move to the next rebalancing date
        current_date += pd.Timedelta(days=rebalancing_frequency)

    # Create graphical representation
    plot_results(positions, pnl_over_time)
    
    return positions, pnl_over_time

def fetch_historical_data(pool_address, start_date, end_date):
    # Replace with actual data fetching logic
    # Example: Fetch data from Uniswap API or local storage
    return pd.DataFrame()  # Placeholder

def get_pool_state(historical_data, current_date):
    # Replace with actual logic to extract pool state from historical data
    pool_state = {
        "current_profit": 500,  # Placeholder value
        "price_out_of_range": False,  # Placeholder value
        "time_since_last_adjustment": 40000,  # Placeholder value
        "pool_volatility": 0.2  # Placeholder value
    }
    return pool_state

def get_inference(pool_state, ddpg_agent_path, ppo_agent_path,pool_address):
    # Call the inference endpoint
    #url = "http://127.0.0.1:8000/inference/"
    url = "http://49.213.164.57:40047/inference/"

    data = {
        "pool_state": pool_state,
        "user_preferences": {
            "risk_tolerance": {"profit_taking": 50, "stop_loss": -500},
            "investment_horizon": 7,
            "liquidity_preference": {"adjust_on_price_out_of_range": True},
            "risk_aversion_threshold": 0.1,
            "user_status": "new_user"
        },
        "pool_id": pool_address,
        "ddpg_agent_path": ddpg_agent_path,
        "ppo_agent_path": ppo_agent_path
    }
    response = requests.post(url, json=data)
    inference_result = response.json()
    
    # Extract tick_lower and tick_upper from the inference result
    tick_lower = inference_result['tick_lower']
    tick_upper = inference_result['tick_upper']
    
    return tick_lower, tick_upper


def simulate_rebalancing(capital, tick_lower, tick_upper, historical_data, current_date):
    # Replace with actual rebalancing logic
    # Calculate trade fees and slippage
    trade_fee = 0.003 * capital  # Example: 0.3% trade fee
    slippage = 0.001 * capital  # Example: 0.1% slippage
    new_capital = capital - trade_fee - slippage
    return new_capital, trade_fee, slippage

def plot_results(positions, pnl_over_time):
    # Extract data for plotting
    dates, tick_lowers, tick_uppers, capitals = zip(*positions)
    pnl_dates, pnls = zip(*pnl_over_time)
    
    # Plot positions
    plt.figure(figsize=(14, 7))
    plt.plot(dates, tick_lowers, label='Tick Lower')
    plt.plot(dates, tick_uppers, label='Tick Upper')
    plt.legend()
    plt.title('Positions Over Time')
    plt.xlabel('Date')
    plt.ylabel('Ticks')
    plt.show()
    
    # Plot PNL
    plt.figure(figsize=(14, 7))
    plt.plot(pnl_dates, pnls, label='PNL')
    plt.legend()
    plt.title('PNL Over Time')
    plt.xlabel('Date')
    plt.ylabel('PNL')
    plt.show()

# Example usage
positions, pnl_over_time = backtest_ilp(
    start_date=pd.Timestamp('2022-01-01'),
    end_date=pd.Timestamp('2022-12-31'),
    pool_address='0xcbcdf9626bc03e24f779434178a73a0b4bad62ed',
    ddpg_agent_path='model_storage/ddpg/ddpg_1',
    ppo_agent_path='model_storage/ppo/lstm_actor_critic_batch_norm',
    starting_capital=100000,
    rebalancing_frequency=7  # Rebalance every 7 days
)


ConnectionError: HTTPConnectionPool(host='49.213.164.57', port=40047): Max retries exceeded with url: /inference/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7efce42eeac0>: Failed to establish a new connection: [Errno 111] Connection refused'))